In [14]:
import pyspark

## Contents: <a id = 'Content'></a>
1. [Problem 1](#problem__1) 


2. Problem 2 - [Linear Regression](#problem__2)

    1. [Acceleration as Dependent Variable](#problem__2_1)
    2. [Horsepower as Dependent Variable](#problem__2_2)


3. Problem 3 - [Decision Tree](#problem__3)

    1. [Horsepower as Dependent Variable](#problem__3_1)
    2. [Acceleration as Dependent Variable](#problem__3_2)

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [4]:
spark

In [5]:
df_pyspark = spark.read.csv('Small_Car_Data.csv')

In [6]:
df_pyspark

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string]

In [7]:
df_pyspark.show()

+----+------------+---------+------------+----------+-------------+--------------------+----------+---+-------+------+
| _c0|         _c1|      _c2|         _c3|       _c4|          _c5|                 _c6|       _c7|_c8|    _c9|  _c10|
+----+------------+---------+------------+----------+-------------+--------------------+----------+---+-------+------+
|null|Acceleration|Cylinders|Displacement|Horsepower| Manufacturer|               Model|Model_Year|MPG| Origin|Weight|
|   1|          12|        8|         307|       130|chevrolet    |chevrolet chevell...|        70| 18|USA    |  3504|
|   2|        11.5|        8|         350|       165|buick        |buick skylark 320...|        70| 15|USA    |  3693|
|   3|          11|        8|         318|       150|plymouth     |plymouth satellit...|        70| 18|USA    |  3436|
|   4|          12|        8|         304|       150|amc          |amc rebel sst    ...|        70| 16|USA    |  3433|
|   5|        10.5|        8|         302|      

In [87]:
#this can be done by the following way also
## df_pyspark = spark.read.csv('Small_Car_Data.csv', header = True, inferSchema=True)
## Infer Schema is used because initially the schema is read as string but as we want integer value for some features, That is why we0 used inferSchema.
df_pyspark = spark.read.option('header', 'true').csv('Small_Car_Data.csv', inferSchema=True)

### Problem_1 EDA <a id = problem__1></a>
- This is EDA, if you want to skip to Linear regression, click [here](#Lin_Reg_Problem_1)

In [88]:
df_pyspark.show()

+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower| Manufacturer|               Model|Model_Year| MPG| Origin|Weight|
+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|  1|        12.0|        8|         307|       130|chevrolet    |chevrolet chevell...|        70|18.0|USA    |  3504|
|  2|        11.5|        8|         350|       165|buick        |buick skylark 320...|        70|15.0|USA    |  3693|
|  3|        11.0|        8|         318|       150|plymouth     |plymouth satellit...|        70|18.0|USA    |  3436|
|  4|        12.0|        8|         304|       150|amc          |amc rebel sst    ...|        70|16.0|USA    |  3433|
|  5|        10.5|        8|         302|       140|ford         |ford torino      ...|        70|17.0|USA    |  3449|
|  6|        10.0|        8|         429|       

In [10]:
# to check the data type of the dataframe
df_pyspark.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: integer (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Model_Year: integer (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Weight: integer (nullable = true)



In [11]:
df_part1 = df_pyspark.select(['Displacement', 'Horsepower'])
df_part1.show()

+------------+----------+
|Displacement|Horsepower|
+------------+----------+
|         307|       130|
|         350|       165|
|         318|       150|
|         304|       150|
|         302|       140|
|         429|       198|
|         454|       220|
|         440|       215|
|         455|       225|
|         390|       190|
|         133|       115|
|         350|       165|
|         351|       153|
|         383|       175|
|         360|       175|
|         383|       170|
|         340|       160|
|         302|       140|
|         400|       150|
|         455|       225|
+------------+----------+
only showing top 20 rows



In [12]:
df_part1.printSchema()

root
 |-- Displacement: integer (nullable = true)
 |-- Horsepower: integer (nullable = true)



In [13]:
df_part1.describe().show()

+-------+------------------+------------------+
|summary|      Displacement|        Horsepower|
+-------+------------------+------------------+
|  count|               100|               100|
|   mean|             207.6|            111.83|
| stddev|111.77448435817251|45.345153316456326|
|    min|                85|                46|
|    max|               455|               225|
+-------+------------------+------------------+



In [23]:
## adding columns in pyspark

## this does not have inplace attribute, therefore we If we nne to implement, we need to assign it to something

## df.withColumn('New_columnName', condition)
## sample condition -- df.['displacement']/2

In [22]:
## Dropping the columns

## this does not have inplace attribute, therefore we If we nne to implement, we need to assign it to something

## df.drop('columnName')

In [24]:
## renaming the df
## df.withColumnRenamed('columnName', 'NewColumnName')

<hr style="border:2px solid gray">

## Linear regression for Problem 1 <a id = 'Lin_Reg_Problem_1'></a>
- Go to [Contents](#Content) 

In [15]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols = ['Displacement'], outputCol='Indexed_displacement')



In [16]:
df_LR_part1 = feature_assembler.transform(df_part1)
df_LR_part1.show()

+------------+----------+--------------------+
|Displacement|Horsepower|Indexed_displacement|
+------------+----------+--------------------+
|         307|       130|             [307.0]|
|         350|       165|             [350.0]|
|         318|       150|             [318.0]|
|         304|       150|             [304.0]|
|         302|       140|             [302.0]|
|         429|       198|             [429.0]|
|         454|       220|             [454.0]|
|         440|       215|             [440.0]|
|         455|       225|             [455.0]|
|         390|       190|             [390.0]|
|         133|       115|             [133.0]|
|         350|       165|             [350.0]|
|         351|       153|             [351.0]|
|         383|       175|             [383.0]|
|         360|       175|             [360.0]|
|         383|       170|             [383.0]|
|         340|       160|             [340.0]|
|         302|       140|             [302.0]|
|         400

In [17]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = df_LR_part1.randomSplit([0.9, 0.1])
regressor = LinearRegression(featuresCol='Indexed_displacement', labelCol = 'Horsepower')
regressor = regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([0.3695])

In [19]:
regressor.intercept

35.345636823327595

In [20]:
LR_pred = regressor.evaluate(test_data)

In [21]:
LR_pred.predictions.show()

+------------+----------+--------------------+------------------+
|Displacement|Horsepower|Indexed_displacement|        prediction|
+------------+----------+--------------------+------------------+
|          91|        67|              [91.0]| 68.96635558057675|
|          91|        68|              [91.0]| 68.96635558057675|
|          97|        75|              [97.0]|  71.1831062678679|
|         107|        75|             [107.0]| 74.87769074668648|
|         151|        90|             [151.0]| 91.13386245348828|
|         200|        81|             [200.0]|109.23732639969936|
|         318|       150|             [318.0]| 152.8334232497587|
|         318|       150|             [318.0]| 152.8334232497587|
|         318|       150|             [318.0]| 152.8334232497587|
|         350|       180|             [350.0]|164.65609358197818|
+------------+----------+--------------------+------------------+



In [22]:
LR_pred.meanAbsoluteError, LR_pred.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-1.9663555805767459|
|-0.9663555805767459|
|  3.816893732132101|
|0.12230925331351727|
|-1.1338624534882769|
|-28.237326399699356|
|-2.8334232497587095|
|-2.8334232497587095|
|-2.8334232497587095|
| 15.343906418021817|
+-------------------+



(6.008727916708469, None)

In [43]:
# r^2 shows that how well our LR model fits the test case.
LR_pred.r2

0.9365572756728744

### Problem 2 <a id = 'problem__2'></a>
- features:
    1. Cylinders
    2. Displacement
    3. Manufacturer
    4. Model Year
    5. Origin
    6. Weight
    

- Target:
    1. [Acceleration](#problem__2_1)
    2. [Horsepower](#problem__2_1)
    
    
    
- Please note that some of those are categorical variables.
- Use test data to assess quality of prediction for both target variables. 
- Which of two target variables is easier to predict, in the sense that predicted values differ less from the original values

- Refer Problem 1 EDA for reference [Problem_1](#problem__1)

## Problem 2 EDA <a id = 'problem__2_EDA'></a>

In [89]:
df_pyspark.show()

+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower| Manufacturer|               Model|Model_Year| MPG| Origin|Weight|
+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|  1|        12.0|        8|         307|       130|chevrolet    |chevrolet chevell...|        70|18.0|USA    |  3504|
|  2|        11.5|        8|         350|       165|buick        |buick skylark 320...|        70|15.0|USA    |  3693|
|  3|        11.0|        8|         318|       150|plymouth     |plymouth satellit...|        70|18.0|USA    |  3436|
|  4|        12.0|        8|         304|       150|amc          |amc rebel sst    ...|        70|16.0|USA    |  3433|
|  5|        10.5|        8|         302|       140|ford         |ford torino      ...|        70|17.0|USA    |  3449|
|  6|        10.0|        8|         429|       

In [90]:
df_pyspark = df_pyspark.withColumn("Model_year", df_pyspark.Model_Year+1900)
df_pyspark.show()

+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower| Manufacturer|               Model|Model_year| MPG| Origin|Weight|
+---+------------+---------+------------+----------+-------------+--------------------+----------+----+-------+------+
|  1|        12.0|        8|         307|       130|chevrolet    |chevrolet chevell...|      1970|18.0|USA    |  3504|
|  2|        11.5|        8|         350|       165|buick        |buick skylark 320...|      1970|15.0|USA    |  3693|
|  3|        11.0|        8|         318|       150|plymouth     |plymouth satellit...|      1970|18.0|USA    |  3436|
|  4|        12.0|        8|         304|       150|amc          |amc rebel sst    ...|      1970|16.0|USA    |  3433|
|  5|        10.5|        8|         302|       140|ford         |ford torino      ...|      1970|17.0|USA    |  3449|
|  6|        10.0|        8|         429|       

In [51]:
from pyspark.ml.feature import StringIndexer

In [106]:

indexer = StringIndexer(inputCols=['Manufacturer', 'Origin'], outputCols=['Mnft_categoryIndex', 'Orgn_categoryIndex'])
df_problem2 = indexer.fit(df_pyspark).transform(df_pyspark)



In [107]:
df_problem2 = df_problem2.drop('Manufacturer','Model', 'Origin')

In [108]:
df_problem2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Displacement: integer (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Model_year: integer (nullable = true)
 |-- MPG: double (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Mnft_categoryIndex: double (nullable = false)
 |-- Orgn_categoryIndex: double (nullable = false)



In [109]:
df_problem2.show()

+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower|Model_year| MPG|Weight|Mnft_categoryIndex|Orgn_categoryIndex|
+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+
|  1|        12.0|        8|         307|       130|      1970|18.0|  3504|               1.0|               0.0|
|  2|        11.5|        8|         350|       165|      1970|15.0|  3693|               9.0|               0.0|
|  3|        11.0|        8|         318|       150|      1970|18.0|  3436|               4.0|               0.0|
|  4|        12.0|        8|         304|       150|      1970|16.0|  3433|               2.0|               0.0|
|  5|        10.5|        8|         302|       140|      1970|17.0|  3449|               0.0|               0.0|
|  6|        10.0|        8|         429|       198|      1970|15.0|  4341|             

In [111]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler


In [112]:
df_problem2.columns

['_c0',
 'Acceleration',
 'Cylinders',
 'Displacement',
 'Horsepower',
 'Model_year',
 'MPG',
 'Weight',
 'Mnft_categoryIndex',
 'Orgn_categoryIndex']

In [115]:
Assembler = VectorAssembler(inputCols = ['Cylinders', 'Displacement', 'Model_year', 'MPG', 'Weight',  'Mnft_categoryIndex', 'Orgn_categoryIndex'], 
                           outputCol = 'Features')

In [116]:
OUTPUT_2 = Assembler.transform(df_problem2)

In [117]:
OUTPUT_2.show()

+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+--------------------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower|Model_year| MPG|Weight|Mnft_categoryIndex|Orgn_categoryIndex|            Features|
+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+--------------------+
|  1|        12.0|        8|         307|       130|      1970|18.0|  3504|               1.0|               0.0|[8.0,307.0,1970.0...|
|  2|        11.5|        8|         350|       165|      1970|15.0|  3693|               9.0|               0.0|[8.0,350.0,1970.0...|
|  3|        11.0|        8|         318|       150|      1970|18.0|  3436|               4.0|               0.0|[8.0,318.0,1970.0...|
|  4|        12.0|        8|         304|       150|      1970|16.0|  3433|               2.0|               0.0|[8.0,304.0,1970.0...|
|  5|        10.5|        8|         302|       140|   

In [118]:
from pyspark.ml.feature import StandardScaler

In [120]:
scaler_2 = StandardScaler(inputCol = 'Features', outputCol = 'Scaled_Features_2', withMean = True, withStd = True)

In [121]:
Scaler_Model_2 = scaler_2.fit(OUTPUT_2)

In [122]:
Scaler_Model_2

StandardScalerModel: uid=StandardScaler_e427d80b8fb0, numFeatures=7, withMean=true, withStd=true

In [123]:
scaled_data_2 = Scaler_Model_2.transform(OUTPUT_2)

In [124]:
scaled_data_2.show()

+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+--------------------+--------------------+
|_c0|Acceleration|Cylinders|Displacement|Horsepower|Model_year| MPG|Weight|Mnft_categoryIndex|Orgn_categoryIndex|            Features|   Scaled_Features_2|
+---+------------+---------+------------+----------+----------+----+------+------------------+------------------+--------------------+--------------------+
|  1|        12.0|        8|         307|       130|      1970|18.0|  3504|               1.0|               0.0|[8.0,307.0,1970.0...|[1.32886743167390...|
|  2|        11.5|        8|         350|       165|      1970|15.0|  3693|               9.0|               0.0|[8.0,350.0,1970.0...|[1.32886743167390...|
|  3|        11.0|        8|         318|       150|      1970|18.0|  3436|               4.0|               0.0|[8.0,318.0,1970.0...|[1.32886743167390...|
|  4|        12.0|        8|         304|       150|      1970|1

In [128]:
scaled_data_2.select('Scaled_Features_2').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------+
|Scaled_Features_2                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------------------+
|[1.3288674316739035,0.8892906155709074,-1.1771828306459147,-0.6671273396275227,0.6099044872969679,-0.744231418570891,-0.5493381302054059]  |
|[1.3288674316739035,1.2739937993691872,-1.1771828306459147,-1.0436808709318621,0.8441161379443599,0.42319041448148675,-0.5493381302054059] |
|[1.3288674316739035,0.9877030579379092,-1.1771828306459147,-0.6671273396275227,0.5256378616672185,-0.30644823117624936,-0.5493381302054059]|
|[1.3288674316739035,0.8624508585617251,-1.1771828306459147,-0.918163027163749,0.5219202164188471,-0.5983036894393439,-0.5493381302054059]  |
|[1.32

###  Acceleration as Dependent Variable <a id = 'problem__2_1'></a>
- For Contents, click [here](#Content)

In [208]:
from pyspark.ml.regression import LinearRegression
train_data_P2, test_data_P2 = scaled_data_2.randomSplit([0.9, 0.1])
regressor_P2 = LinearRegression(featuresCol='Scaled_Features_2', labelCol = 'Acceleration', regParam = 0.03)
regressor_P2 = regressor_P2.fit(train_data_P2)

In [209]:
regressor_P2.coefficients

DenseVector([-0.5086, -3.5595, 0.3166, -0.3847, 1.6949, 0.0179, -0.3314])

In [210]:
LR_pred_P2 = regressor_P2.evaluate(test_data_P2)
LR_pred_P2.predictions

DataFrame[_c0: int, Acceleration: double, Cylinders: int, Displacement: int, Horsepower: int, Model_year: int, MPG: double, Weight: int, Mnft_categoryIndex: double, Orgn_categoryIndex: double, Features: vector, Scaled_Features_2: vector, prediction: double]

In [211]:
LR_pred_P2.meanAbsoluteError, LR_pred_P2.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.7855603546912864|
| -0.8381193382910297|
|   1.578052823758025|
|  0.7088714336786808|
|-0.43618233044278654|
+--------------------+



(0.8693572561723617, None)

In [212]:
LR_pred_P2.r2

0.9243420899057848

### Horsepower as Dependent Variable <a id = 'problem__2_2'></a>

- With Acceleration as a targert variable it is easier to predict by independent features as the mean absolute error is less than error while Horsepower being the dependent variable

- For Contents, click [here](#Content)

In [224]:
from pyspark.ml.regression import LinearRegression
train_data_P2_hp, test_data_P2_hp = scaled_data_2.randomSplit([0.9, 0.1])
regressor_P2_hp = LinearRegression(featuresCol='Scaled_Features_2', labelCol = 'Horsepower', regParam = 0.03)
regressor_P2_hp = regressor_P2_hp.fit(train_data_P2_hp)

In [225]:
regressor_P2_hp.coefficients

DenseVector([-8.9065, 35.5243, -5.2915, -3.4768, 11.384, 4.7333, 0.064])

In [226]:
LR_pred_P2_hp = regressor_P2_hp.evaluate(test_data_P2_hp)
LR_pred_P2_hp.predictions

DataFrame[_c0: int, Acceleration: double, Cylinders: int, Displacement: int, Horsepower: int, Model_year: int, MPG: double, Weight: int, Mnft_categoryIndex: double, Orgn_categoryIndex: double, Features: vector, Scaled_Features_2: vector, prediction: double]

In [227]:
LR_pred_P2_hp.meanAbsoluteError, LR_pred_P2_hp.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 14.834814981179278|
|-14.718073816324107|
|-16.481021460424145|
| -8.857428880130925|
| -15.68701014301547|
| 22.133919025068778|
|  5.426620289483068|
|-0.9474620557972884|
| 2.8410563490636207|
|  8.015118801141824|
+-------------------+



(10.99425258016285, None)

In [237]:
LR_pred_P2_hp.r2

0.9266323397748596

In [ ]:
LR_pred_P2_hp.

## Decision Tree <a id = 'problem__3'></a>
### Horsepower as Dependent Variable <a id = 'problem__3_1'></a>
- For Contents, click [here](#Content)

In [266]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [267]:
train_data_P2_hp, test_data_P2_hp = scaled_data_2.randomSplit([0.9, 0.1])

In [241]:
from pyspark.ml.regression import DecisionTreeRegressor

In [268]:
DTR = DecisionTreeRegressor(featuresCol='Scaled_Features_2', labelCol = 'Horsepower')
DTR_model = DTR.fit(train_data_P2_hp)

In [269]:
DTR_model_pred = DTR_model.transform(test_data_P2_hp)

In [270]:
DTR_model_pred

DataFrame[_c0: int, Acceleration: double, Cylinders: int, Displacement: int, Horsepower: int, Model_year: int, MPG: double, Weight: int, Mnft_categoryIndex: double, Orgn_categoryIndex: double, Features: vector, Scaled_Features_2: vector, prediction: double]

In [254]:
DTR_model_pred.summary()

DataFrame[summary: string, _c0: string, Acceleration: string, Cylinders: string, Displacement: string, Horsepower: string, Model_year: string, MPG: string, Weight: string, Mnft_categoryIndex: string, Orgn_categoryIndex: string, prediction: string]

In [294]:
train_data_P2_hp, test_data_P2_hp = scaled_data_2.randomSplit([0.8, 0.2])
DTR_model_pred = DTR_model.transform(test_data_P2_hp)
DTR_model_pred.select("prediction", "Horsepower").show(5)
evaluator = RegressionEvaluator(
    labelCol="Horsepower", predictionCol="prediction", metricName="r2")
evaluator_rsme = RegressionEvaluator(
    labelCol="Horsepower", predictionCol="prediction", metricName="rmse")
r2 = evaluator.evaluate(DTR_model_pred)
rsme = evaluator_rsme.evaluate(DTR_model_pred)

print ("R2 Score  of test data = %g" % r2)
print ("RMSE  of test data = %g" % rmse)

+----------+----------+
|prediction|Horsepower|
+----------+----------+
|     220.0|       220|
|     225.0|       225|
|     180.0|       190|
|     115.0|       115|
|     180.0|       175|
+----------+----------+
only showing top 5 rows

R2 Score  of test data = 0.992422
RMSE  of test data = 5.0521


### Accelerations as Dependent Variable <a id = 'problem__3_2'></a>
- For Contents, click [here](#Content)

In [287]:
train_data_P2, test_data_P2 = scaled_data_2.randomSplit([0.8, 0.2])

In [288]:
DTR = DecisionTreeRegressor(featuresCol='Scaled_Features_2', labelCol = 'Acceleration')
DTR_model_a = DTR.fit(train_data_P2)

In [295]:
DTR_model_pred_a = DTR_model_a.transform(test_data_P2)
DTR_model_pred_a.select("prediction", "Acceleration").show(5)
evaluator = RegressionEvaluator(
    labelCol="Acceleration", predictionCol="prediction", metricName="r2")
evaluator_rsme = RegressionEvaluator(
    labelCol="Acceleration", predictionCol="prediction", metricName="rmse")
r2_a = evaluator.evaluate(DTR_model_pred_a)
rsme_a = evaluator_rsme.evaluate(DTR_model_pred_a)

print ("R2 Score  of test data = %g" % r2)
print ("RMSE  of test data = %g" % rmse)

+-----------------+------------+
|       prediction|Acceleration|
+-----------------+------------+
|             10.8|        12.0|
|9.428571428571429|        11.5|
|16.62857142857143|        17.5|
|            11.75|         8.0|
|9.428571428571429|         9.5|
+-----------------+------------+
only showing top 5 rows

R2 Score  of test data = 0.992422
RMSE  of test data = 5.0521
